In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip -q "/content/drive/MyDrive/Colab Notebooks/Monkeypox Data/Custom Dataset.zip"

In [3]:
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model
import xgboost as xgb
import numpy as np

In [4]:
BATCH_SIZE = 32
IMG_SIZE = (224, 224)

In [5]:
mobilenet = MobileNet(input_shape=(*IMG_SIZE, 3), include_top=False, weights='imagenet')

17225924/17225924 [==============================] - 2s 0us/step


In [6]:
for layer in mobilenet.layers:
    layer.trainable = False

In [7]:
model = Sequential()
model.add(mobilenet)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [8]:
model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [9]:
train_dir = '/content/train'
validation_dir = '/content/val'
test_dir = '/content/test'

train_datagen = ImageDataGenerator(rescale=1./255,
                rotation_range=20,
                horizontal_flip=True,
                zoom_range=0.2)

val_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
                train_dir,
                target_size=IMG_SIZE,
                class_mode='binary',
                batch_size=32)

val_generator = val_datagen.flow_from_directory(
                validation_dir,
                target_size=IMG_SIZE,
                class_mode='binary',
                batch_size=32)

test_generator = test_datagen.flow_from_directory(
                test_dir,
                target_size=IMG_SIZE,
                class_mode='binary',
                batch_size=32)

Found 1772 images belonging to 2 classes.
Found 94 images belonging to 2 classes.
Found 288 images belonging to 2 classes.


In [10]:
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

In [11]:
history = model.fit(train_generator,
                    epochs=100,
                    validation_data=val_generator,
                    callbacks=[early_stopping])

Epoch 1/100
56/56 [==============================] - 32s 396ms/step - loss: 0.8751 - accuracy: 0.5604 - val_loss: 0.4800 - val_accuracy: 0.7447
Epoch 2/100
56/56 [==============================] - 19s 347ms/step - loss: 0.6674 - accuracy: 0.6755 - val_loss: 0.3821 - val_accuracy: 0.8191
Epoch 3/100
56/56 [==============================] - 19s 347ms/step - loss: 0.5801 - accuracy: 0.7286 - val_loss: 0.3432 - val_accuracy: 0.8191
Epoch 4/100
56/56 [==============================] - 21s 375ms/step - loss: 0.5636 - accuracy: 0.7421 - val_loss: 0.3180 - val_accuracy: 0.8404
Epoch 5/100
56/56 [==============================] - 19s 344ms/step - loss: 0.4927 - accuracy: 0.7709 - val_loss: 0.3128 - val_accuracy: 0.8511
Epoch 6/100
56/56 [==============================] - 21s 372ms/step - loss: 0.4846 - accuracy: 0.7619 - val_loss: 0.3026 - val_accuracy: 0.8511
Epoch 7/100
56/56 [==============================] - 20s 348ms/step - loss: 0.4950 - accuracy: 0.7641 - val_loss: 0.2990 - val_accuracy:

In [12]:
train_features = mobilenet.predict(train_generator)
val_features = mobilenet.predict(val_generator)
test_features = mobilenet.predict(test_generator)

9/9 [==============================] - 1s 72ms/step


In [13]:
print(train_features.shape)
print(val_features.shape)
print(test_features.shape)

(1772, 7, 7, 1024)
(94, 7, 7, 1024)
(288, 7, 7, 1024)


In [14]:
x_train = train_features.reshape(train_features.shape[0], -1)
x_train.shape

(1772, 50176)

In [15]:
x_val = val_features.reshape(val_features.shape[0], -1)
x_val.shape

(94, 50176)

In [16]:
x_test = test_features.reshape(test_features.shape[0], -1)
x_test.shape

(288, 50176)

In [17]:
xgboost = xgb.XGBClassifier(objective='binary:logistic', random_state=42)

In [18]:
xgboost.fit(x_train, train_generator.classes)

XGBClassifier(random_state=42)

In [19]:
val_accuracy = xgboost.score(x_val, val_generator.classes)
print(f'Validation accuracy: {val_accuracy * 100}%')

Validation accuracy: 56.38297872340425%


In [20]:
predictions = xgboost.predict(x_test)
accuracy = sum(predictions == test_generator.classes) / len(predictions)
print(f'Accuracy: {accuracy * 100}%')

Accuracy: 54.513888888888886%


In [21]:
print(f'True labels: {test_generator.classes}')
print(f'Predictions: {predictions}')

True labels: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
Predictions: [1 1 1 0 1 0 1 1 1 1 0 1 1 0 1 1 0 0 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1
 1 0 1 1 0 1 0 0 1 0 1 1 1 1 0 0 0 1 1 1 0 1 0 0 1 1 1 1 1 1 1 1 0 0 1 1 1
 1 0 1 0 1 1 0 1 0 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 0 0 0 1 1 1 1 0
 1 1 1 1 1 0 1 0 1 1 1 0 1 1 1 0 1 1 1 1 1 1 0 0 0 1 0 1 1 1 1 1 1 1 1 1 1
 1 0 1 1 0 0 0 0 1 1 0 0 0 0 1 1 1 0 1 1 1 0 1 0 0 1 1 1 0 1 1 0 0 0 0 0 1
 1 0 1 0 1 0 1